In [1]:
import os
import json
import time
from datetime import datetime
import jax
import random
import jax.numpy as jnp
from flax import linen as nn

In [2]:
folder = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/JAX_MODULES-Easy_Multidigit_Addition_Decimal/'
folder_decision = f'{folder}DECISION_MODULE/'
folder_unit = f'{folder}UNIT_MODULE/' 
folder_carry = f'{folder}CARRY_MODULE/' 
folder_no_modules = f'{folder}NO_MODULES/' 

In [3]:
# Clase que define la estructura del modelo
class carry_LSTMModel(nn.Module):
    @nn.compact
    def __call__(self, x):
        lstm_1 = nn.LSTMCell(features=16)
        dense = nn.Dense(2)

        carry1 = lstm_1.initialize_carry(jax.random.PRNGKey(0), (x.shape[0],)) 

        for t in range(x.shape[1]):  # Iterar sobre los pasos temporales
            carry1, x_t = lstm_1(carry1, x[:, t])

        hidden_state = carry1[0] 
        final_output = nn.softmax(dense(hidden_state))
        return final_output

class unit_LSTMModel(nn.Module):
    @nn.compact
    def __call__(self, x):
        lstm_1 = nn.LSTMCell(features=16)
        lstm_2 = nn.LSTMCell(features=32)
        lstm_3 = nn.LSTMCell(features=16)
        dense = nn.Dense(10)

        carry1 = lstm_1.initialize_carry(jax.random.PRNGKey(0), (x.shape[0],))
        carry2 = lstm_2.initialize_carry(jax.random.PRNGKey(1), (x.shape[0],))
        carry3 = lstm_3.initialize_carry(jax.random.PRNGKey(2), (x.shape[0],))

        for t in range(x.shape[1]):  # Iterar sobre los pasos temporales
            carry1, x_t = lstm_1(carry1, x[:, t])
            carry2, x_t = lstm_2(carry2, x_t)
            carry3, x_t = lstm_3(carry3, x_t)

        hidden_state = carry3[0]  # Estado oculto tiene forma (batch_size, 32)
        final_output = nn.softmax(dense(hidden_state))
        return final_output

In [4]:
def init_carry_params(rng, input_shape=(1, 1, 2)):
    """Inicializa los parámetros del modelo LSTM unit_carry_model desde cero."""
    lstm_model = carry_LSTMModel()  # Usa la misma clase de tu implementación
    params = lstm_model.init(rng, jnp.ones(input_shape))["params"]
    return params

def init_unit_params(rng, input_shape=(1, 1, 2)):
    """Inicializa los parámetros del modelo LSTM unit_carry_model desde cero."""
    lstm_model = unit_LSTMModel()  # Usa la misma clase de tu implementación
    params = lstm_model.init(rng, jnp.ones(input_shape))["params"]
    return params

# Función para crear parámetros entrenables (v_0, ..., v_11)
def init_params(rng, epsilon=0.1, param_type='AP'):
    """Inicializa parámetros entrenables y del módulo unit_carry_model."""
    # Inicializar los parámetros v_0_1_{i}_{j}, ..., v_1_2_{i}_{j}
    v_values_init = {f'v_{k}_{n}_{i}_{j}': 0 for k in range(2) for n in range(1, 3) for i in [0, 1] for j in [2, 3]}
    
    if param_type == 'AP':
        v_values_init['v_0_1_1_3'] = 1
        v_values_init['v_1_1_0_2'] = 1
        v_values_init['v_1_2_1_3'] = 1
    
    # Aplicar un ligero ruido aleatorio
    v_params = {
        key: random.uniform(-10, 10) * epsilon + value
        for key, value in v_values_init.items()
    }
    
    # Inicializar los parámetros del módulo unit_carry_model
    carry_params = init_carry_params(rng)
    unit_params = init_unit_params(rng)

    # Retornar todos los parámetros
    return {'decision_params': v_params, 'carry_params': carry_params, 'unit_params': unit_params}

In [5]:
# Crear la carpeta de destino si no existe
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Guardar los parámetros en un archivo JSON
def save_params_to_file(params, filename):
    with open(filename, 'w') as f:
        json.dump(params, f, default=lambda o: o.tolist() if isinstance(o, jnp.ndarray) else o)

# Generar y guardar los parámetros en 10 archivos con intervalos de 2 segundos
def generate_and_save_params(num_files=10, epsilon=0.1, param_type='AP'):
    if param_type == 'AP':
        base_dir = f"{folder_no_modules}Parameters/AP_{epsilon}/"
        decision_dir = f"{folder_decision}Parameters/AP_{epsilon}/"
        carry_dir = f"{folder_carry}Parameters/AP_{epsilon}/"
        unit_dir = f"{folder_unit}Parameters/AP_{epsilon}/"
    elif param_type == 'RP':
        base_dir = f"{folder_no_modules}Parameters/RP_{epsilon}/"
        decision_dir = f"{folder_decision}Parameters/RP_{epsilon}/"
        carry_dir = f"{folder_carry}Parameters/RP_{epsilon}/"
        unit_dir = f"{folder_unit}Parameters/RP_{epsilon}/"
    else:
        print('Error en param_type')
        return
        
    create_directory(base_dir)
    create_directory(decision_dir)
    create_directory(carry_dir)
    create_directory(unit_dir)
    
    rng = jax.random.PRNGKey(10)
    
    for i in range(num_files):
        rng, subkey = jax.random.split(rng)
        v_params = init_params(subkey, epsilon, param_type)

        timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        
        filename_decision = os.path.join(decision_dir, f"trainable_model_{timestamp}.json")
        save_params_to_file(v_params['decision_params'], filename_decision)

        filename_carry = os.path.join(carry_dir, f"trainable_model_{timestamp}.json")
        save_params_to_file(v_params['carry_params'], filename_carry)

        filename_unit = os.path.join(unit_dir, f"trainable_model_{timestamp}.json")
        save_params_to_file(v_params['unit_params'], filename_unit)

        filename = os.path.join(base_dir, f"trainable_model_{timestamp}.json")
        save_params_to_file(v_params, filename)

        print(f"Archivo guardado: {filename}")
        print(f"Archivo guardado en folder_decision: {filename_decision}")
        print(f"Archivo guardado en folder_carry: {filename_carry}")
        print(f"Archivo guardado en folder_unit: {filename_unit}")

        time.sleep(2)

In [6]:
generate_and_save_params(num_files=10, epsilon=0.2, param_type='AP')

Archivo guardado: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/JAX_MODULES-Easy_Multidigit_Addition_Decimal/NO_MODULES/Parameters/AP_0.2/trainable_model_2025_01_24_02_47_02.json
Archivo guardado en folder_decision: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/JAX_MODULES-Easy_Multidigit_Addition_Decimal/DECISION_MODULE/Parameters/AP_0.2/trainable_model_2025_01_24_02_47_02.json
Archivo guardado en folder_carry: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/JAX_MODULES-Easy_Multidigit_Addition_Decimal/CARRY_MODULE/Parameters/AP_0.2/trainable_model_2025_01_24_02_47_02.json
Archivo guardado en folder_unit: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/JAX_MODULES-Easy_Multidigit_Addition_Decimal/UNIT_MODULE/Parameters/AP_0.2/trainable_model_2025_01_24_02_47_02.json
Archivo guardado: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Do

In [15]:
generate_and_save_params(num_files=10, epsilon=0.1)
generate_and_save_params(num_files=10, epsilon=0.2)
generate_and_save_params(num_files=10, epsilon=0.3)
generate_and_save_params(num_files=10, epsilon=0.5)
generate_and_save_params(num_files=10, epsilon=0.7)
generate_and_save_params(num_files=10, epsilon=1)
generate_and_save_params(num_files=10, epsilon=1.5)
generate_and_save_params(num_files=10, epsilon=2)
generate_and_save_params(num_files=10, epsilon=3)
generate_and_save_params(num_files=10, epsilon=5)

Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_52_53.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_52_55.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_52_57.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_52_59.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_53_01.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_53_03.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_53_05.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_53_07.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_53_09.json
Archivo guardado: Parameters/AP_0.1/trainable_model_2025_01_15_14_53_11.json
Archivo guardado: Parameters/AP_0.2/trainable_model_2025_01_15_14_53_13.json
Archivo guardado: Parameters/AP_0.2/trainable_model_2025_01_15_14_53_15.json
Archivo guardado: Parameters/AP_0.2/trainable_model_2025_01_15_14_53_17.json

In [5]:
generate_and_save_params(num_files=15, epsilon=0.1)
generate_and_save_params(num_files=15, epsilon=0.2)
generate_and_save_params(num_files=15, epsilon=0.3)
generate_and_save_params(num_files=15, epsilon=0.4)
generate_and_save_params(num_files=15, epsilon=0.5)
generate_and_save_params(num_files=15, epsilon=0.6)
generate_and_save_params(num_files=15, epsilon=0.7)
generate_and_save_params(num_files=15, epsilon=0.8)
generate_and_save_params(num_files=15, epsilon=0.9)
generate_and_save_params(num_files=15, epsilon=1)
generate_and_save_params(num_files=15, epsilon=1.2)
generate_and_save_params(num_files=15, epsilon=1.5)
generate_and_save_params(num_files=15, epsilon=1.7)
generate_and_save_params(num_files=15, epsilon=2)
generate_and_save_params(num_files=15, epsilon=2.5)
generate_and_save_params(num_files=15, epsilon=3)
generate_and_save_params(num_files=15, epsilon=3.5)
generate_and_save_params(num_files=15, epsilon=4)
generate_and_save_params(num_files=15, epsilon=4.5)
generate_and_save_params(num_files=15, epsilon=5)
generate_and_save_params(num_files=15, epsilon=5.5)
generate_and_save_params(num_files=15, epsilon=6)
generate_and_save_params(num_files=15, epsilon=6.5)
generate_and_save_params(num_files=15, epsilon=7)
generate_and_save_params(num_files=15, epsilon=7.5)
generate_and_save_params(num_files=15, epsilon=8)
generate_and_save_params(num_files=15, epsilon=8.5)
generate_and_save_params(num_files=15, epsilon=9)
generate_and_save_params(num_files=15, epsilon=9.5)
generate_and_save_params(num_files=15, epsilon=10)

Archivo guardado: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition_Decimal/Parameters/AP_0.05\trainable_model_2024_12_17_12_17_19.json
Archivo guardado: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition_Decimal/Parameters/AP_0.05\trainable_model_2024_12_17_12_17_21.json
Archivo guardado: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition_Decimal/Parameters/AP_0.05\trainable_model_2024_12_17_12_17_23.json
Archivo guardado: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition_Decimal/Parameters/AP_0.05\trainable_model_2024_12_17_12_17_25.json
Archivo guardado: D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition_Decimal/Parameters/AP_0.05\trainable_model_2024_12_17_12_17_27.json
Archivo guardado: D:/OneDrive - Universidad Complu